## PH model
@author: Or Duek

In this notebook there are two different ways of calculating the Hybrid model
On each function you'll see reference to the relevant paper

In [3]:
%config Completer.use_jedi = False

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import theano
import theano.tensor as tt
import scipy
import pingouin as pg
import os

import pymc3 as pm
import arviz as az
import statsmodels.api as sm
import statsmodels.formula.api as smf

from glob import glob

#### Load data (Nachshon)

In [4]:
#Load Age and MoCA scores from the session log file.
age = pd.read_excel('/media/Data/Lab_Projects/Aging/aging_session_log.xlsx', sheet_name='sessions').iloc[:,[0,7,9]]
moca = pd.read_excel('/media/Data/Lab_Projects/Aging/aging_session_log.xlsx', sheet_name='assessments').iloc[:,[0,22]]

# Change name of the first column to 'sub'
age.columns.values[0] = moca.columns.values[0] = "sub"

# remove unnecessary rows
age = age[age.gender != 'gender']
age = age.dropna().reset_index()
age = age.drop(['index'], axis=1)

print("Number of participants with age: ",age.shape[0],'; females: ', age[age['gender']=='F'].shape[0])
# merge files
age = age.merge(moca, left_on='sub', right_on='sub')
age['sub'] = age['sub'].map(lambda x: int(x.lstrip('AG_'))) # remove the AG_ prfix from the subject number
age['sub'] = age['sub'].astype(int)
age['age'] = age.age.astype('int')

print('Number of participants with MoCA: ', age.shape[0], '; MoCA > 25:', age[age.moca_score>25].shape[0],'; females: ', age[age['gender']=='F'].shape[0])
print('mean:', np.mean(age.age), 'std:', np.std(age.age), 'min-max:',np.min(age.age),np.max(age.age))
age_c = age[age['moca_score']>25]
print('mean:', np.mean(age_c.age), 'std:', np.std(age_c.age), 'min-max:',np.min(age_c.age),np.max(age_c.age))

Number of participants with age:  91 ; females:  43
Number of participants with MoCA:  81 ; MoCA > 25: 63 ; females:  38
mean: 54.45679012345679 std: 22.041514916632877 min-max: 18 89
mean: 51.888888888888886 std: 21.998637123089434 min-max: 18 88


In [5]:
glober = '/media/Data/Lab_Projects/Aging/behavioral/Reversal/AG_*_RV/ETLearning_*.csv'

db = pd.DataFrame()

for sub in glob(glober):
    
    try:
        df = pd.read_csv(sub)
        df['sub'] = sub.split('_')[2]
        if df.shape[0] == 70:
            db = pd.concat([db, df], axis = 0)
            #db = db.append(df)#[df.trialNum<36])
    except:
        print(sub)
        print('error')

#db['rating'] = db['rating'].replace(0, np.nan)
db = db.sort_values(by=['sub','trialNum'])
print('number of subject: ', len(db['sub'].unique()))

number of subject:  88


In [6]:
db['sub'] = db['sub'].astype('int')
db = db.merge(age_c, left_on='sub', right_on='sub')
#db = db[db.moca_score >25]
db = db.sort_values('sub')
print('Valid subjects: ', len(np.unique(db['sub'])))

Valid subjects:  63


In [7]:
x = db[db['RT']==-1].groupby('sub').count()['rating']
x = x[x>5].reset_index()
print(x)
db = db[~db['sub'].isin(x['sub'].values)]

   sub  rating
0  102      13


In [8]:
subs = db.drop_duplicates('sub')
subs = subs.drop(['trialNum', 'rectOri', 'rectValue', 'rating', 'RT'],axis = 1).reset_index()
#subs.head()

## get descriptive data

In [99]:
n_subj   = len(db['sub'].unique())
n_trials = max(db.trialNum)

trials, subj = np.meshgrid(range(n_trials), range(n_subj))
trials = tt.as_tensor_variable(trials.T)
subj   = tt.as_tensor_variable(subj.T)

In [100]:
stim   = np.reshape([db['rectOri']],   (n_subj, n_trials)).T
reward = np.reshape([db['rectValue']], (n_subj, n_trials)).T
rating = np.reshape([db['rating']],    (n_subj, n_trials)).T

stim   = np.array(stim/45,  dtype='int')
reward = np.array(reward/6, dtype='int')

In [101]:
rating = np.array(rating/9)

In [96]:
rating[0,:]

array([0.33333333, 1.        , 0.77777778, 0.33333333, 0.66666667,
       0.77777778, 0.33333333, 0.77777778, 0.22222222, 0.88888889,
       0.33333333, 0.22222222, 0.33333333, 0.88888889, 0.77777778,
       0.33333333, 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.55555556, 0.77777778, 0.55555556, 1.        ,
       0.55555556, 0.11111111, 0.33333333, 0.88888889, 0.11111111,
       0.44444444, 0.44444444, 1.        , 0.77777778, 0.77777778,
       0.11111111, 0.55555556, 0.44444444, 0.88888889, 0.22222222,
       0.        , 1.        , 0.11111111, 0.22222222, 0.44444444,
       0.22222222, 0.22222222, 1.        , 1.        , 1.        ,
       0.11111111, 0.11111111, 0.55555556, 0.55555556, 0.55555556,
       1.        , 0.44444444, 0.88888889, 0.77777778, 0.33333333,
       0.11111111, 0.11111111])

In [102]:
stim = tt.as_tensor_variable(stim)
reward = tt.as_tensor_variable(reward)

In [60]:
# Rouhani et al. version https://elifesciences.org/articles/61077
# generate functions to run
def update_Q_r(stim, shock,
             Qs,vec,
             eta,kappa,n_subj):
    """
    This function updates the Q table according to Hybrid PH model
    For information, please see this paper: https://www.sciencedirect.com/science/article/pii/S0896627316305840?via%3Dihub
  
    """
      
    delta = shock - Qs[tt.arange(n_subj), stim]
    #alpha = tt.set_subtensor(alpha[tt.arange(n_subj), stim], eta * abs(delta) + (1-eta)*alpha[tt.arange(n_subj), stim])
    # based on Rohani et al. 
    #alpha = tt.set_subtensor(alpha[tt.arange(n_subj), stim], eta + kappa * abs(delta))
    alpha = eta + (kappa * abs(delta))
    alpha = 1 / (1 + pm.math.exp(-alpha)) # sigmoid function
    Qs = tt.set_subtensor(Qs[tt.arange(n_subj),stim], Qs[tt.arange(n_subj),stim] + alpha * delta)
    
    # in order to get a vector of expected outcome (dependent on the stimulus presentes [CS+, CS-] 
    # we us if statement (switch in theano)
    vec = tt.set_subtensor(vec[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                Qs[tt.arange(n_subj),1], Qs[tt.arange(n_subj),0])))
    
    # we use the same idea to get the associability per trial
    # assoc = tt.set_subtensor(assoc[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
    #                                                             alpha[tt.arange(n_subj),1], alpha[tt.arange(n_subj),0])))
    
    return Qs, vec, alpha

In [62]:
az.summary(tr)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
eta[0],-2.955,0.554,-3.960,-1.910,0.013,0.010,1789.0,1901.0,1.00
eta[1],-3.230,0.469,-4.107,-2.343,0.010,0.007,2091.0,2053.0,1.00
eta[2],-3.743,0.520,-4.749,-2.801,0.011,0.008,2310.0,1692.0,1.00
eta[3],-3.087,0.612,-4.226,-1.904,0.014,0.010,1875.0,1489.0,1.00
eta[4],-3.651,0.513,-4.589,-2.676,0.012,0.009,1753.0,1559.0,1.01
...,...,...,...,...,...,...,...,...,...
"expected_value[69, 57]",4.374,0.813,2.792,5.800,0.015,0.011,3143.0,2363.0,1.00
"expected_value[69, 58]",6.231,0.561,5.202,7.277,0.009,0.007,3648.0,2742.0,1.00
"expected_value[69, 59]",2.147,0.751,0.626,3.519,0.023,0.017,1168.0,642.0,1.00
"expected_value[69, 60]",5.491,0.771,4.036,6.887,0.014,0.010,2943.0,3319.0,1.00


In [103]:
# generate functions to run
def update_Q_hall(stim, reward,
             Qs,vec,alpha,assoc,
             eta,kappa, n_subj):
    """
    This function updates the Q table according to Hybrid PH model
    For information, please see this paper: https://www.sciencedirect.com/science/article/pii/S0896627316305840?via%3Dihub
  
    """
      
    delta = reward - Qs[tt.arange(n_subj), stim]
    alpha = tt.set_subtensor(alpha[tt.arange(n_subj), stim], eta * abs(delta) + (1-eta)*alpha[tt.arange(n_subj), stim])
    Qs = tt.set_subtensor(Qs[tt.arange(n_subj),stim], Qs[tt.arange(n_subj),stim] + kappa*alpha[tt.arange(n_subj), stim] * delta)
    
    # in order to get a vector of expected outcome (dependent on the stimulus presentes [CS+, CS-] 
    # we us if statement (switch in theano)
    vec = tt.set_subtensor(vec[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                Qs[tt.arange(n_subj),1], Qs[tt.arange(n_subj),0])))
    
    # we use the same idea to get the associability per trial
    assoc = tt.set_subtensor(assoc[tt.arange(n_subj),0], (tt.switch(tt.eq(stim,1), 
                                                                alpha[tt.arange(n_subj),1], alpha[tt.arange(n_subj),0])))
    
    return Qs, vec, alpha, assoc

In [116]:
def theano_llik_td(eta, kappa, stim, reward,
              n_subj):
   

    Qs = 0.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 0.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    alpha = 0 * tt.ones((n_subj,2), dtype='float64') # vector to save the relevant stimulus's expactation
    assoc = 0 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec, alpha, assoc], updates = theano.scan(
        fn=update_Q_hall,
        sequences=[stim, reward],
        outputs_info=[Qs, vec, alpha, assoc],
        non_sequences=[eta, kappa, n_subj])
    
   
    return Qs, vec, alpha, assoc

In [117]:
# test the function first
results, vec, alpha, assoc = theano_llik_td(0.2,0.5, stim, reward, n_subj)

In [130]:
vec.eval()[:,0,0]

array([0.475     , 0.475     , 0.4334375 , 0.4334375 , 0.38431007,
       0.50519618, 0.42848486, 0.33469333, 0.28892263, 0.24896591,
       0.35807444, 0.48366839, 0.2152228 , 0.59114523, 0.18725487,
       0.45776025, 0.35417522, 0.16428161, 0.14545892, 0.5127974 ,
       0.13001024, 0.6322642 , 0.46825855, 0.11727363, 0.34916111,
       0.26592492, 0.10670724, 0.20813844, 0.09787711, 0.16762287,
       0.13870998, 0.09043958, 0.11764531, 0.30269665, 0.46831308,
       0.35739818, 0.2284625 , 0.19550822, 0.27690811, 0.21935006,
       0.1691252 , 0.17806337, 0.14800668, 0.13103091, 0.11729102,
       0.26925715, 0.22492321, 0.14808024, 0.38709244, 0.12593992,
       0.30731534, 0.10928984, 0.09653635, 0.2472026 , 0.42167438,
       0.08659223, 0.3257101 , 0.25580154, 0.07870659, 0.07235309,
       0.0671571 , 0.20533507, 0.39390633, 0.54570044, 0.40658648,
       0.06284782, 0.30713521, 0.49072145, 0.05922662, 0.0561458 ])

In [132]:
alpha.eval()[:,0,:]

array([[0.        , 0.1       ],
       [0.1       , 0.1       ],
       [0.1       , 0.175     ],
       [0.175     , 0.175     ],
       [0.175     , 0.2266875 ],
       [0.2533125 , 0.2266875 ],
       [0.30368924, 0.2266875 ],
       [0.30368924, 0.25821201],
       [0.30368924, 0.27350828],
       [0.30368924, 0.27659115],
       [0.32864836, 0.27659115],
       [0.3913038 , 0.27659115],
       [0.3913038 , 0.2710661 ],
       [0.41630936, 0.2710661 ],
       [0.41630936, 0.25989744],
       [0.45127654, 0.25989744],
       [0.45257328, 0.25989744],
       [0.45257328, 0.24536893],
       [0.45257328, 0.22915146],
       [0.49122358, 0.22915146],
       [0.49122358, 0.21241296],
       [0.49041938, 0.21241296],
       [0.51878835, 0.21241296],
       [0.51878835, 0.19593241],
       [0.50868239, 0.19593241],
       [0.47677813, 0.19593241],
       [0.47677813, 0.18020066],
       [0.43460749, 0.18020066],
       [0.43460749, 0.16550197],
       [0.38931368, 0.16550197],
       [0.

In [ ]:
with pm.Model() as m2:
  
      
    eta = pm.Beta('eta', 2,2, shape=n_subj)
    kappa = pm.Beta('kappa', 2,2, shape=n_subj)
    beta = pm.Normal('beta',0, 1, shape=n_subj)
    eps = pm.HalfNormal('eps', 5)
    
    # intercept
    intercept = pm.Normal('intercept', 0, 5, shape=n_subj)
    
    Qs = 0.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 0.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    alpha = 0 * tt.ones((n_subj,2), dtype='float64') # vector to save the relevant stimulus's expactation
    assoc = 0 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    
    [Qs,vec, alpha, assoc], updates = theano.scan(
        fn=update_Q_hall,
        sequences=[stim, reward],
        outputs_info=[Qs, vec, alpha, assoc],
        non_sequences=[eta, kappa, n_subj])
    
    vec_ = vec[trials, subj,0] * beta[subj] + intercept[subj]
    
    scrs = pm.Normal('scrs', mu = vec_, sd = eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
 #   ev = pm.Deterministic('expected_value', vec_)
    # add associability
   # alpha = pm.Deterministic('pe', alpha)
    
    tr = pm.sample(target_accept=.95, chains=4, cores=10, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 10 jobs)
NUTS: [intercept, eps, beta, kappa, eta]


In [136]:
tr

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data

In [135]:
az.summary(tr)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],0.080,0.313,-0.481,0.664,0.012,0.010,1003.0,494.0,1.01
beta[1],0.790,0.259,0.384,1.311,0.009,0.007,1129.0,822.0,1.00
beta[2],0.066,0.302,-0.443,0.648,0.010,0.009,1268.0,785.0,1.00
beta[3],-0.295,0.285,-0.818,0.249,0.007,0.005,1661.0,1346.0,1.00
beta[4],0.222,0.220,-0.176,0.648,0.005,0.004,2025.0,1600.0,1.00
...,...,...,...,...,...,...,...,...,...
kappa[58],0.465,0.213,0.079,0.849,0.004,0.003,2940.0,1623.0,1.00
kappa[59],0.415,0.231,0.035,0.823,0.004,0.003,2465.0,1701.0,1.00
kappa[60],0.434,0.209,0.055,0.802,0.003,0.003,3324.0,2260.0,1.00
kappa[61],0.562,0.172,0.253,0.878,0.003,0.002,3112.0,2297.0,1.00


## Hierarchical one

In [139]:
with pm.Model() as m2_h:
  
    # hyperpriors for eta and kappa
    phi = pm.Uniform("phi", lower=0.0, upper=1.0, shape=2)
    
    # κ   
    k_log1 = pm.Exponential("k_log1", lam=1.5)
    k1 = pm.Deterministic("k1", tt.exp(k_log1))
    kappa = pm.Beta("kappa", alpha=phi[0] * k1, beta=(1.0 - phi[0]) * k1, shape=n_subj)
    
    # β (with reparametarization)
    beta_h = pm.Normal('beta_h', 0,1)
    beta_sd = pm.HalfNormal('beta_sd', 5)
    b_matt = pm.Normal('beta_matt', mu=0, sigma=1, shape=n_subj)
    beta = pm.Deterministic('beta',beta_h + beta_sd*b_matt)
    
    # η
    k_log2 = pm.Exponential("k_log2", lam=1.5)
    k2 = pm.Deterministic("k2", tt.exp(k_log2))
    eta = pm.Beta('η', alpha=phi[1] * k2, beta=(1.0 - phi[1]) * k2, shape=n_subj)
    
    eps = pm.HalfNormal('eps', 5)
    
    Qs = 0.5 * tt.ones((n_subj,2), dtype='float64') # set values for boths stimuli (CS+, CS-)
    vec = 0.5 * tt.ones((n_subj,1), dtype='float64') # vector to save the relevant stimulus's expactation
    alpha = 0 * tt.ones((n_subj,2), dtype='float64')
    assoc = 0 * tt.ones((n_subj,1), dtype='float64')
    
    [Qs,vec, alpha, assoc], updates = theano.scan(
        fn=update_Q_hall,
        sequences=[stim, reward],
        outputs_info=[Qs, vec, alpha, assoc],
        non_sequences=[eta, kappa, n_subj])
   
    
    vec_ = vec[trials, subj,0] * beta[subj]
    
    scrs = pm.Normal('scrs', vec_, eps, observed=rating) 
    
    # add matrix of expected values (trials X subjects)
    ev = pm.Deterministic('expected_value', vec_)
    # add associabillity
    #assoc = pm.Deterministic('alpha', assoc)
    
    trH = pm.sample(target_accept=.95, chains=4, cores=4, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, η, k_log2, beta_matt, beta_sd, beta_h, kappa, k_log1, phi]


/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/pymc3/step_methods/hmc/integration.py:108: RuntimeWarning: invalid value encountered in double_scalars
  energy = kinetic - logp
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 9754 seconds.
There were 7 divergences after tuning. Increase `target_accept` or reparameterize.
There were 39 divergences a

In [140]:
az.compare({'nonH':tr, 'Hierarchical': trH})

/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/arviz/stats/stats.py:695: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
Hierarchical,0,-641.177079,72.622249,0.000000,0.615088,36.332558,0.000000,True,log
nonH,1,-655.083378,136.303019,13.906299,0.384912,38.167451,10.600253,False,log
